In [33]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_rows', 10)

from LeNetModel import *
import theano.tensor as T

from scipy.misc import imread, imresize

from ipywidgets import FloatProgress
from IPython.display import display

from time import time
import datetime

import pickle

In [11]:
df = pd.read_csv('Data/driver_imgs_list.csv')
path = "/Users/johnstamford/Desktop/StateFarmDriver Project/Data/"
img = imread(path + "train/c5/img_274.jpg", mode="F")

# Model
## Randomness

In [12]:
rng = np.random.RandomState(123)

## Settings

In [57]:
df.classname.value_counts()

c0    2489
c3    2346
c4    2326
c6    2325
c2    2317
c5    2312
c1    2267
c9    2129
c7    2002
c8    1911
Name: classname, dtype: int64

In [62]:
x = T.matrix('x')
layer0_input = x.reshape((1, 1, 480, 640))

nkerns=[30, 20, 10, 5, 1]

# Which am I selecting?
classList = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

## CNN Layers

In [63]:
layer0 = LeNetConvPoolLayer(
        rng,
        input=layer0_input,
        image_shape=(1, 1, 480, 640),
        filter_shape=(nkerns[0], 1, 5, 5),
        poolsize=(2, 2)
    )

layer0_test = theano.function([x], layer0.output)

layer1 = LeNetConvPoolLayer(
        rng,
        input=layer0.output,
        image_shape=(1, nkerns[0], 238, 318),
        filter_shape=(nkerns[1], nkerns[0], 5, 5),
        poolsize=(2, 2)
    )

layer1_test = theano.function([x], layer1.output)

layer2 = LeNetConvPoolLayer(
        rng,
        input=layer1.output,
        image_shape=(1, nkerns[1], 117, 157),
        filter_shape=(nkerns[2], nkerns[1], 5, 5),
        poolsize=(2, 2)
    )

layer2_test = theano.function([x], layer2.output)

layer3 = LeNetConvPoolLayer(
        rng,
        input=layer2.output,
        image_shape=(1, nkerns[2], 56, 76),
        filter_shape=(nkerns[3], nkerns[2], 5, 5),
        poolsize=(2, 2)
    )

layer3_test = theano.function([x], layer3.output)

layer4 = LeNetConvPoolLayer(
        rng,
        input=layer3.output,
        image_shape=(1, nkerns[3], 26, 36),
        filter_shape=(nkerns[4], nkerns[3], 5, 5),
        poolsize=(2, 2)
    )

layer4_test = theano.function([x], layer4.output)

# Data
Select which data to use

In [64]:
new_n_feat = 176 # Based on the output from above

In [65]:
# Get the data to loop through
#data = df[(df.classname == 'c0') | (df.classname == 'c1')]

data = df[df['classname'].isin(classList)]
data

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg
...,...,...,...
22419,p081,c9,img_56936.jpg
22420,p081,c9,img_46218.jpg
22421,p081,c9,img_25946.jpg
22422,p081,c9,img_67850.jpg


In [66]:
# A new dataframe to store the new data
newData = pd.DataFrame(columns=range(1,new_n_feat))
newData['Class'] = np.nan
newData.shape

(0, 176)

In [67]:
# Create a subset of the main dataset
data = data.copy(deep=True)

data.shape

(22424, 3)

In [68]:
len(data.index.values)

22424

In [69]:
df.shape

(22424, 3)

# Run the model

In [70]:
maxCount = data.shape[0]
t0 = time()

f = FloatProgress(min=0, max=maxCount)
display(f)

count = 0



for i in data.index.values:
    
    #print(data.iloc[i].img)
    img_name = df.iloc[i].img
    cla = df.iloc[i].classname
    
    img = imread(path + "train/" + cla + "/" + img_name, mode="F")
    
    img_layer4 = layer4_test(img)
    
    # Add the new row
    # Add the new row
    value = img_layer4.reshape([1,176])
    value = pd.DataFrame(value)
    value['Class'] = cla
    value.shape

    newData = newData.append(value)
    
    count = count + 1
    f.value = count

t1 = time()
print("Completed Time %i in %.2f sec" % ( maxCount, (t1 - t0)))

Completed Time 22424 in 9187.37 sec


# Save the data

In [72]:
ts = time()
st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H-%M-%S')

#newData.to_csv("Data/firstOutput " + st + ".csv", index=False)

#newData.to_csv("Data/firstOutput ALL.csv", index=False)